# Solución etapa 3 - FE + Training

In [3]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl

In [4]:
# Cargar los datasets
import pandas as pd

historicos_ordenes_cleaned = pd.read_csv('../data/cleaned/historicos_ordenes_cleaned.csv')
registros_condiciones_cleaned = pd.read_csv('../data/cleaned/registros_condiciones_cleaned.csv')
caracteristicas_equipos_cleaned = pd.read_csv('../data/cleaned/caracteristicas_equipos_cleaned.csv')


## Feature eng

### Target

In [5]:
# Generar target

target_column="Fallo"

def code_target(val):
    if val == 'Correctivo':
        return 1
    else:
        return 0


historicos_ordenes_cleaned[target_column] = historicos_ordenes_cleaned["Tipo_Mantenimiento"].apply(code_target)
historicos_ordenes_cleaned.drop("Tipo_Mantenimiento", axis=1, inplace=True)

### historicos_ordenes

In [6]:
tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1
 4           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1
(9951, 7)


In [7]:
historicos_ordenes_cleaned['Fecha'] = pd.to_datetime(historicos_ordenes_cleaned['Fecha'])

historicos_ordenes_cleaned['Año'] = historicos_ordenes_cleaned['Fecha'].dt.year
historicos_ordenes_cleaned['Mes'] = historicos_ordenes_cleaned['Fecha'].dt.month
historicos_ordenes_cleaned['Día'] = historicos_ordenes_cleaned['Fecha'].dt.day

tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo    Año    Mes    Día
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------  -----  -----  -----
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0   2020      1      1
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0   2020      1      1
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0   2020      1      1
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1   2020      1      1
 4           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1   2020      1      1
(9951, 10)


In [8]:
# Tiempo medio entre fallos por fecha

# Filtrar solo los registros con fallos
fallos = historicos_ordenes_cleaned[historicos_ordenes_cleaned['Fallo'] == 1].copy()

# Ordenar por fecha
fallos = fallos.sort_values(by=['ID_Equipo', 'Fecha'])

# Calcular los intervalos entre fallos consecutivos
fallos['Intervalo'] = fallos.groupby('ID_Equipo')['Fecha'].diff()
fallos['Intervalo_Horas'] = fallos['Intervalo'].dt.total_seconds() / 3600

# Calcular el Tiempo medio entre fallos por equipo
mtbf_por_idequipo = fallos.groupby('ID_Equipo')['Intervalo_Horas'].mean().reset_index()
mtbf_por_idequipo.rename(columns={'Intervalo_Horas': 'Tiempo_medio_entre_fallos(h)'}, inplace=True)

tabl(mtbf_por_idequipo)

      ID_Equipo    Tiempo_medio_entre_fallos(h)
--  -----------  ------------------------------
 0            1                        1164
 1            2                         667.462
 2            3                         984.167
 3            4                        1300.43
 4            5                        1341.71
(499, 2)


In [9]:
# Mezclar historicos_ordenes_cleaned + mtbf_por_idequipo

historicos_mtbf = pd.merge(historicos_ordenes_cleaned, mtbf_por_idequipo, on=['ID_Equipo'], how='left')
tabl(historicos_mtbf)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo    Año    Mes    Día    Tiempo_medio_entre_fallos(h)
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------  -----  -----  -----  ------------------------------
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0   2020      1      1                        1220.12
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0   2020      1      1                         772.5
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0   2020      1      1                         752
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1   2020      1      1                         699.077
 4         

In [10]:
# Mezclar historicos_ordenes_cleaned + historicos_mtbf

historicos_historicos_mtbf = pd.merge(historicos_ordenes_cleaned, historicos_mtbf, on=['ID_Equipo','Año','Mes','Día'], how='inner')
tabl(historicos_historicos_mtbf)

      ID_Orden_x    ID_Equipo  Fecha_x                Costo_Mantenimiento_x    Duracion_Horas_x  Ubicacion_x      Fallo_x    Año    Mes    Día    ID_Orden_y  Fecha_y                Costo_Mantenimiento_y    Duracion_Horas_y  Ubicacion_y      Fallo_y    Tiempo_medio_entre_fallos(h)
--  ------------  -----------  -------------------  -----------------------  ------------------  -------------  ---------  -----  -----  -----  ------------  -------------------  -----------------------  ------------------  -------------  ---------  ------------------------------
 0             1           36  2020-01-01 00:00:00                  6754.12                   3  Planta Norte           0   2020      1      1             1  2020-01-01 00:00:00                  6754.12                   3  Planta Norte           0                        1220.12
 1             2          244  2020-01-01 01:00:00                  4274.63                  35  Planta Norte           0   2020      1      1             2  

In [11]:
# Verificamos nulos
print(historicos_historicos_mtbf.isna().sum())
historicos_historicos_mtbf.dropna(inplace=True)

ID_Orden_x                      0
ID_Equipo                       0
Fecha_x                         0
Costo_Mantenimiento_x           0
Duracion_Horas_x                0
Ubicacion_x                     0
Fallo_x                         0
Año                             0
Mes                             0
Día                             0
ID_Orden_y                      0
Fecha_y                         0
Costo_Mantenimiento_y           0
Duracion_Horas_y                0
Ubicacion_y                     0
Fallo_y                         0
Tiempo_medio_entre_fallos(h)    0
dtype: int64


In [12]:
# Eliminar columnas sin valor de historicos_mtbf
historicos_mtbf_clean = historicos_mtbf.drop(columns=['ID_Orden','Costo_Mantenimiento','Duracion_Horas','Fecha'])
tabl(historicos_mtbf_clean)

      ID_Equipo  Ubicacion       Fallo    Año    Mes    Día    Tiempo_medio_entre_fallos(h)
--  -----------  ------------  -------  -----  -----  -----  ------------------------------
 0           36  Planta Norte        0   2020      1      1                        1220.12
 1          244  Planta Norte        0   2020      1      1                         772.5
 2          297  Planta Norte        0   2020      1      1                         752
 3          431  Planta Sur          1   2020      1      1                         699.077
 4          403  Planta Norte        1   2020      1      1                        1368.86
(9951, 7)


### registros_condiciones

In [13]:
registros_condiciones_cleaned.drop(columns=['ID_Registro'], index=1, inplace=True)

registros_condiciones_cleaned['Fecha']=pd.to_datetime(registros_condiciones_cleaned['Fecha'])
registros_condiciones_cleaned['Año'] = registros_condiciones_cleaned['Fecha'].dt.year
registros_condiciones_cleaned['Mes'] = registros_condiciones_cleaned['Fecha'].dt.month
registros_condiciones_cleaned['Día'] = registros_condiciones_cleaned['Fecha'].dt.day

tabl(registros_condiciones_cleaned)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    Año    Mes    Día
--  -----------  -------------------  ---------------  ----------------  ------------------  -----  -----  -----
 0          260  2020-01-01 00:00:00           101.43              1.22               71849   2020      1      1
 2          443  2020-01-01 02:00:00            76.04              2.93               33106   2020      1      1
 3          281  2020-01-01 03:00:00            97.09              3.25               31744   2020      1      1
 4          427  2020-01-01 04:00:00           139.42              4.22               78104   2020      1      1
 5           48  2020-01-01 05:00:00            98.77              2.73               89664   2020      1      1
(8999, 8)


In [14]:
# Agrupar por 'ID_Equipo', 'Año' y 'Mes', y calcular los promedios
promedios_por_equipo_mes = (
    registros_condiciones_cleaned.groupby(['ID_Equipo', 'Año', 'Mes','Día'])[['Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']]
      .mean()
      .reset_index()
)

# Renombrar las columnas para claridad
promedios_por_equipo_mes.rename(columns={
    'Temperatura_C': 'Promedio_Temperatura_C',
    'Vibracion_mm_s': 'Promedio_Vibracion_mm_s',
    'Horas_Operativas': 'Promedio_Horas_Operativas'
}, inplace=True)

tabl(promedios_por_equipo_mes)

      ID_Equipo    Año    Mes    Día    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas
--  -----------  -----  -----  -----  ------------------------  -------------------------  ---------------------------
 0            1   2020      1      2                     93.8                        0.7                         80054
 1            1   2020      1     21                    102.99                       2.31                        74797
 2            1   2020      3      2                    145.97                       1.57                        95240
 3            1   2020      3     16                    147.18                       2.49                         6872
 4            1   2020      4     17                     58.93                       3.54                        28186
(8793, 7)


In [15]:
promedios_por_equipo_mes.isna().sum()
promedios_por_equipo_mes.dropna(inplace=True)
promedios_por_equipo_mes.shape

(8755, 7)

### Unir merged_hitorico_registro + promedios + caracteristicas

In [16]:
# Unir datasets
final_data = pd.merge(historicos_mtbf_clean, promedios_por_equipo_mes, on=['ID_Equipo','Año','Mes'], how='inner')
final_data = pd.merge(final_data, caracteristicas_equipos_cleaned, on='ID_Equipo', how='inner')

tabl(final_data)

      ID_Equipo  Ubicacion       Fallo    Año    Mes    Día_x    Tiempo_medio_entre_fallos(h)    Día_y    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  ------------  -------  -----  -----  -------  ------------------------------  -------  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0           36  Planta Norte        0   2020      1        1                         1220.12       13                    139.97                       4.31                        41114  Motor          Siemens       M400               4351                           3549
 1          297  Planta Norte        0   2020      1        1                          752           1                     71.11                       3.24                        42929  Tran

In [17]:
# Limpiar columnas accesorias
final_data.drop(columns=['ID_Equipo','Año', 'Mes', 'Día_x', 'Día_y'], inplace=True)

tabl(final_data)

    Ubicacion       Fallo    Tiempo_medio_entre_fallos(h)    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  ------------  -------  ------------------------------  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0  Planta Norte        0                         1220.12                    139.97                       4.31                        41114  Motor          Siemens       M400               4351                           3549
 1  Planta Norte        0                          752                        71.11                       3.24                        42929  Transformador  GE            X100               3728                           8016
 2  Planta Norte        0                          752                        76.25                 

### Duplicados

In [18]:
# Eliminar duplicados

print(headr('Duplicados final_data:'),final_data.duplicated().sum())
# final_data.drop_duplicates(inplace=True)
# tabl(final_data)


Duplicados final_data:
 1141


In [19]:
# Verificamos nulos
print(headr('Nulos final_data:'),final_data.isna().sum())


Nulos final_data:
 Ubicacion                       0
Fallo                           0
Tiempo_medio_entre_fallos(h)    0
Promedio_Temperatura_C          0
Promedio_Vibracion_mm_s         0
Promedio_Horas_Operativas       0
Tipo_Equipo                     0
Fabricante                      0
Modelo                          0
Potencia_kW                     0
Horas_Recomendadas_Revision     0
dtype: int64


In [20]:
columnas_con_nulos = final_data.columns[final_data.isna().any()].tolist()
columnas_con_nulos

[]

In [21]:
# Imputamos nulos
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# final_data[columnas_con_nulos] = imputer.fit_transform(final_data[columnas_con_nulos])

In [22]:
tabl(final_data)

    Ubicacion       Fallo    Tiempo_medio_entre_fallos(h)    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  ------------  -------  ------------------------------  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0  Planta Norte        0                         1220.12                    139.97                       4.31                        41114  Motor          Siemens       M400               4351                           3549
 1  Planta Norte        0                          752                        71.11                       3.24                        42929  Transformador  GE            X100               3728                           8016
 2  Planta Norte        0                          752                        76.25                 

In [23]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12729 entries, 0 to 12728
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ubicacion                     12729 non-null  object 
 1   Fallo                         12729 non-null  int64  
 2   Tiempo_medio_entre_fallos(h)  12729 non-null  float64
 3   Promedio_Temperatura_C        12729 non-null  float64
 4   Promedio_Vibracion_mm_s       12729 non-null  float64
 5   Promedio_Horas_Operativas     12729 non-null  float64
 6   Tipo_Equipo                   12729 non-null  object 
 7   Fabricante                    12729 non-null  object 
 8   Modelo                        12729 non-null  object 
 9   Potencia_kW                   12729 non-null  float64
 10  Horas_Recomendadas_Revision   12729 non-null  int64  
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


### Volvemos a explorar con columnas codificadas


In [24]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(final_data)
categorical_columns = categorical_columns_selector(final_data)

print(headr("Numerical columns"), numerical_columns)
print(headr("Categorical columns"), categorical_columns)


Numerical columns
 ['Fallo', 'Tiempo_medio_entre_fallos(h)', 'Promedio_Temperatura_C', 'Promedio_Vibracion_mm_s', 'Promedio_Horas_Operativas', 'Potencia_kW', 'Horas_Recomendadas_Revision']

Categorical columns
 ['Ubicacion', 'Tipo_Equipo', 'Fabricante', 'Modelo']


In [25]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

final_data_coded =final_data.copy()
final_data_coded[categorical_columns] =final_data_coded[categorical_columns].apply(LabelEncoder().fit_transform)   
tabl(final_data_coded)

      Ubicacion    Fallo    Tiempo_medio_entre_fallos(h)    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas    Tipo_Equipo    Fabricante    Modelo    Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------  ------------------------------  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0            1        0                         1220.12                    139.97                       4.31                        41114              2             3         0           4351                           3549
 1            1        0                          752                        71.11                       3.24                        42929              3             1         1           3728                           8016
 2            1        0                          752                        76.25                      

In [26]:
# Evaluamos la aportación de cada columna
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2

X= final_data_coded.drop(target_column, axis=1)
y = final_data_coded[target_column]

fvalue_selector = SelectKBest(f_classif, k=2)

X_kbest = fvalue_selector.fit(X,y)

feature_scores = pd.DataFrame({"Feature": X.columns,"Score": X_kbest.scores_}).sort_values(by="Score", ascending=False)

print(headr("Feature scores"))
round(feature_scores,2)


Feature scores



,Feature,Score
1,Tiempo_medio_entre_fallos(h),248.15
7,Modelo,7.30
6,Fabricante,7.04
9,Horas_Recomendadas_Revision,5.31
3,Promedio_Vibracion_mm_s,2.78
5,Tipo_Equipo,1.55
4,Promedio_Horas_Operativas,0.36
0,Ubicacion,0.24
8,Potencia_kW,0.16
2,Promedio_Temperatura_C,0.06


> Una característica tiene demasiado peso y determina toda la clasificación

In [27]:
# Eliminamos columnas sin peso

columns_to_drop = feature_scores[feature_scores['Score'] == 0]['Feature']
final_data.drop(columns=columns_to_drop, inplace=True)
tabl(final_data)

    Ubicacion       Fallo    Tiempo_medio_entre_fallos(h)    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  ------------  -------  ------------------------------  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0  Planta Norte        0                         1220.12                    139.97                       4.31                        41114  Motor          Siemens       M400               4351                           3549
 1  Planta Norte        0                          752                        71.11                       3.24                        42929  Transformador  GE            X100               3728                           8016
 2  Planta Norte        0                          752                        76.25                 

## Exportar 

In [28]:
final_data.to_csv('../data/preprocessed/preprocessed_data_v1.csv', index=False)